In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from collections import defaultdict

import sys 
sys.path.append(r'C:\Users\jcmar\my_files\SportsBetting\FeatureEngineering\ufc_features')
sys.path.append(r'C:\Users\jcmar\my_files\SportsBetting\FeatureEngineering')

from ufc_features import single_event_features, apply_rolling_stats, non_rolling_stats, upcoming_event_features


c:\Users\jcmar\my_files\SportsBetting\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
def standardize_col_names(ufc_df):
    ufc_data = {
        'height_red': ufc_df['height_blue'],
        'height_blue': ufc_df['height_red'],
        
        'reach_red': ufc_df['reach_blue'],
        'reach_blue': ufc_df['reach_red'],
        
        'stance_red': ufc_df['stance_blue'],
        'stance_blue': ufc_df['stance_red'],
        
        'dob_red': ufc_df['dob_blue'],
        'dob_blue': ufc_df['dob_red'],
        
        'kd_red': ufc_df['kd_blue'],
        'kd_blue': ufc_df['kd_red'],
        
        'sub_red': ufc_df['sub_blue'],
        'sub_blue': ufc_df['sub_red'],
        
        'sig_str_red': ufc_df['sig_str_blue'],
        'sig_str_blue': ufc_df['sig_str_red'],
        
        'sig_str_percent_red': ufc_df['sig_str_percent_blue'],
        'sig_str_percent_blue': ufc_df['sig_str_percent_red'],
        
        'td_red': ufc_df['td_blue'],
        'td_blue': ufc_df['td_red'],
        
        'sub_att_red': ufc_df['sub_att_blue'],
        'sub_att_blue': ufc_df['sub_att_red'],
        
        'rev_red': ufc_df['rev_blue'],
        'rev_blue': ufc_df['rev_red'],
        
        'ctrl_red': ufc_df['ctrl_blue'],
        'ctrl_blue': ufc_df['ctrl_red'],
        
        'head_red': ufc_df['head_blue'],
        'head_blue': ufc_df['head_red'],
        
        'body_red': ufc_df['body_blue'],
        'body_blue': ufc_df['body_red'],
        
        'leg_red': ufc_df['leg_blue'],
        'leg_blue': ufc_df['leg_red'],
        
        'distance_red': ufc_df['distance_blue'],
        'distance_blue': ufc_df['distance_red'],
        
        'clinch_red': ufc_df['clinch_blue'],
        'clinch_blue': ufc_df['clinch_red'],
        
        'ground_red': ufc_df['ground_blue'],
        'ground_blue': ufc_df['ground_red'],
        
        'td_pct_red': ufc_df['td_pct_blue'],
        'td_pct_blue': ufc_df['td_pct_red'],
        
        'total_strikes_red': ufc_df['total_strikes_blue'],
        'total_strikes_blue': ufc_df['total_strikes_red'],
        
        'red_record': ufc_df['blue_record'],
        'blue_record': ufc_df['red_record'],
        'event_date':ufc_df['event_date'],
        'event_name': ufc_df['event_name'],
        'event_location':ufc_df['event_location'],
        'round': ufc_df['round'],
        'method': ufc_df['method'],
        'weight_class': ufc_df['weight_class'],
        'red_fighter': ufc_df['red_fighter'],
        'blue_fighter': ufc_df['blue_fighter']
    }

    return pd.DataFrame(ufc_data) 


winners_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\winners.csv')
past_stats = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\scraped_data_ufc.csv') #use past stats pre feature engineering 
past_stats = standardize_col_names(past_stats)
past_stats = pd.concat([winners_df, past_stats], axis=1)

past_stats['event_date'] = pd.to_datetime(past_stats['event_date'])
past_stats = past_stats.sort_values(by='event_date', ascending=True).reset_index(drop=True)
# most_recent_date = past_stats.iloc[-1]['event_date']
# most_recent_date = '2025-03-01'
past_stats = past_stats.loc[:, ~past_stats.columns.str.lower().str.contains('unnamed')]

In [40]:
def standardize_dates(stats, odds):
    for i, row in stats.iterrows():

        #find all matchups between red and blue fighter in odds
        mask = (odds['red_clean'] == row['red_clean']) & (odds['blue_clean'] == row['blue_clean']) 

        # if odds contains the stats matchup 
        if mask.any():
            for i in odds[mask].index:

                # iterate and find dates of each red/blue matchup 
                target_date = odds.at[i, 'date']

                # check if current stats date matches with +- odds date 
                if (row['date'] - pd.Timedelta(days=1) == target_date) or (row['date'] + pd.Timedelta(days=1) == target_date):

                    # if found, replace odds_date with stats date for standardization 
                    odds.at[i, 'date'] = row['date']
    return odds 

def clean_col(col):
    col = col.str.lower() \
        .str.replace('-', ' ', regex=False) \
        .str.replace('.', '', regex=False) \
        .str.replace("'", '', regex=False) \
        .str.replace(r'\bsaint\b', 'st', case=False, regex=True)
    return col 

def standardized_merge(stats_df, odds_df):
    odds_df = odds_df.loc[:, ~odds_df.columns.str.contains('^Unnamed')]

    stats = stats_df.reset_index(drop=True).copy()
    odds = odds_df.reset_index(drop=True).copy()
    
    odds['date'] = pd.to_datetime(odds['event_date'])
    stats['date'] = pd.to_datetime(stats['event_date'])

    odds = odds.dropna(subset=['date'])

    stats['red_clean'] = clean_col(stats['red_fighter'])
    stats['blue_clean'] = clean_col(stats['blue_fighter'])

    stats['red_fighter_stats'] = stats['red_fighter']
    stats['blue_fighter_stats'] = stats['blue_fighter']

    odds['red_clean'] = clean_col(odds['red_fighter'])
    odds['blue_clean'] = clean_col(odds['blue_fighter'])

    odds['red_fighter_odds'] = odds['red_fighter']
    odds['blue_fighter_odds'] = odds['blue_fighter']

    
    odds = standardize_dates(stats, odds)

    # print(odds.groupby(['red_clean', 'blue_clean','date']).size().reset_index(name='count').query('count>1'))

    print(stats.groupby(['red_clean', 'blue_clean','date']).size().reset_index(name='count').query('count>1'))
    
    # merge odds onto stats after standardizing names and dates to match stats 
    new_df = pd.merge(stats, odds, on=['red_clean', 'blue_clean','date'], how='left')

    print(new_df.groupby(['red_clean', 'blue_clean','date']).size().reset_index(name='count').query('count>1'))
    new_df = new_df.loc[:, ~new_df.columns.str.contains('^Unnamed')]

    new_df = new_df.rename(columns={'red_clean': 'red_fighter', 'blue_clean':'blue_fighter'})

    new_df = new_df.drop(columns=['red_fighter_x','red_fighter_y','blue_fighter_x','blue_fighter_y','event_date_y','event_date_x'])
    new_df = new_df.drop_duplicates()

    return new_df

def stats_preprocessing(stats_df):
    
    stats_df = stats_df.loc[:, ~stats_df.columns.str.contains('^Unnamed')]
    past_event_stats = single_event_features(stats_df.copy())
    # past_event_stats.to_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_singe_event_features.csv', save_index=False)

    return past_event_stats

def apply_rolling(df):
    rolling_df = apply_rolling_stats(df)
    return rolling_df

def merge_with_odds(odds_df, stats_df):
    merged_df = standardized_merge(stats_df, odds_df)
    return merged_df

test_pre = stats_preprocessing(past_stats)
test_rolling = apply_rolling(test_pre)

odds_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\entire_odds_history.csv')
odds_stats = merge_with_odds(odds_df, test_rolling)


KeyError: 'performance_bonus_winner'

In [ ]:
duplicates = odds_df.groupby(['red_fighter', 'blue_fighter','event_date']).size().reset_index(name='count').query('count>1')
print(duplicates)

In [5]:
duplicates = odds_stats.groupby(['red_fighter', 'blue_fighter','date']).size().reset_index(name='count').query('count>1')
print(duplicates)
dup_keys = duplicates[['red_fighter', 'blue_fighter', 'date']]

duplicate_rows = odds_stats.merge(
    dup_keys,
    on=['red_fighter', 'blue_fighter', 'date'],
    how='inner'
)

for name, group in duplicate_rows.groupby(['red_fighter', 'blue_fighter', 'date']):
    print(f"\nGroup: {name}")
    # print(group)

    group_no_index = group.drop(columns=['red_fighter', 'blue_fighter', 'date'])
    
    # Find which columns have more than one unique value
    diff_cols = group_no_index.nunique()[group_no_index.nunique() > 1].index.tolist()

    if len(diff_cols) == 0:
        print("✅ All rows in this group are exactly the same.")
    else:
        print("⚠️ These columns differ between rows:", diff_cols)

             red_fighter          blue_fighter       date  count
382       anderson silva           demian maia 2010-04-10      2
644   ashlee evans smith            andrea lee 2019-02-17      2
645   ashlee evans smith         ketlen vieira 2017-04-15      2
907        brendan allen         jacob malkoun 2022-06-11      2
914      brendan oreilly           alan jouban 2016-03-19      2
998       bryce mitchell               dan ige 2023-09-23      2
1251       chris barnett          martin buday 2022-04-16      2
1457         cody durden       charles johnson 2023-04-29      2
1788       danny roberts         claudio silva 2019-03-16      2
2211        dustin ortiz       hector sandoval 2017-08-05      2
2212        dustin ortiz       matheus nicolau 2018-07-28      2
3515          jim miller         gleison tibau 2010-09-15      2
3682      john castaneda         daniel marcos 2024-06-08      2
3818           jon jones  vladimir matyushenko 2010-08-01      2
3821          jon madsen 

In [8]:
odds_stats_drop = odds_stats.drop(columns=['og_blue_name', 'og_red_fighter'])
odds_stats_drop = odds_stats_drop.drop_duplicates()

duplicates = odds_stats_drop[odds_stats_drop.duplicated(subset=['red_fighter','blue_fighter','date'], keep=False)]
print(duplicates[['red_fighter','blue_fighter','date']])


        red_fighter  blue_fighter       date
6387  chris barnett  martin buday 2022-04-16
6388  chris barnett  martin buday 2022-04-16


In [5]:
target_date = pd.Timestamp('2007-07-07')
odds_stats['date'] = pd.to_datetime(odds_stats['date'], errors='coerce')

result = odds_stats[odds_stats['date'] == target_date]
result[['red_fighter','blue_fighter', 'date', 'red_sig_str_attempted', 'sub_att_pr_red', 'winner']].head(15)

,red_fighter,blue_fighter,date,red_sig_str_attempted,sub_att_pr_red,winner
566,antonio rodrigo nogueira,heath herring,2007-07-07,120,NaN,1
567,chris lytle,jason gilliam,2007-07-07,8,0.500000,1
568,jorge gurgel,diego saraiva,2007-07-07,230,0.166667,1
569,stephan bonnar,mike nickels,2007-07-07,5,0.555556,1
570,frankie edgar,mark bocek,2007-07-07,54,0.000000,1
571,sean sherk,hermes franca,2007-07-07,55,0.057143,1
572,anderson silva,nate marquardt,2007-07-07,15,0.333333,1
573,kenny florian,alvin robinson,2007-07-07,32,0.522222,1
574,rashad evans,tito ortiz,2007-07-07,64,0.000000,1


In [ ]:
pd.set_option('display.max_rows', None)
print(test_pre.isna().sum())

print(test_rolling.isna().sum())

In [7]:
duplicates = test_pre[test_pre.duplicated(subset=['red_fighter','blue_fighter','event_date'], keep=False)]
print(duplicates[['red_fighter','blue_fighter','event_date']])

duplicates = test_rolling[test_rolling.duplicated(subset=['red_fighter','blue_fighter','event_date'], keep=False)]
print(duplicates[['red_fighter','blue_fighter','event_date']])


duplicates = odds_stats[odds_stats.duplicated(subset=['red_fighter','blue_fighter','date'], keep=False)].reset_index(drop=True)
print(duplicates[['red_fighter','blue_fighter','date', 'winner']])

diff_cols = duplicates.columns[duplicates.iloc[0] != duplicates.iloc[1]]
print(diff_cols)

print(duplicates.drop(columns=['og_blue_name', 'og_red_fighter']).drop_duplicates().shape, duplicates.shape)

Empty DataFrame
Columns: [red_fighter, blue_fighter, event_date]
Index: []
Empty DataFrame
Columns: [red_fighter, blue_fighter, event_date]
Index: []
        red_fighter       blue_fighter       date  winner
0   michael bisping        matt hamill 2007-09-08       1
1   michael bisping        matt hamill 2007-09-08       1
2    anderson silva        demian maia 2010-04-10       1
3    anderson silva        demian maia 2010-04-10       1
4        jon madsen      karlos vemola 2010-07-03       1
..              ...                ...        ...     ...
61   john castaneda      daniel marcos 2024-06-08       0
62   mauricio ruffy      james llontop 2024-11-16       1
63   mauricio ruffy      james llontop 2024-11-16       1
64    kevin holland  reinier de ridder 2025-01-18       0
65    kevin holland  reinier de ridder 2025-01-18       0

[66 rows x 4 columns]
Index(['og_blue_name', 'og_red_fighter'], dtype='object')
(34, 59) (66, 61)


In [142]:
test_pre[['red_fighter','blue_fighter','winner','red_wins', 'red_losses', 'blue_wins','blue_losses', 'event_date', 'height_red','reach_red', 'reach_blue', 'sigstrikes_pm_red','sigstrikes_pm_blue','red_sig_str_attempted']].tail()

,red_fighter,blue_fighter,winner,red_wins,red_losses,blue_wins,blue_losses,event_date,height_red,reach_red,reach_blue,sigstrikes_pm_red,sigstrikes_pm_blue,red_sig_str_attempted
7852,Danny Barlow,Sam Patterson,Sam Patterson,13,2,9,1,2025-03-01,75.0,78.0,79.0,3.000000,3.600000,30
7853,Hyder Amil,William Gomis,Hyder Amil,14,3,11,0,2025-03-01,72.0,73.0,70.0,7.666667,6.733333,202
7854,Nasrat Haqparast,Esteban Ribovics,Nasrat Haqparast,14,2,18,5,2025-03-01,70.0,69.0,72.0,9.000000,9.800000,273
7855,Cody Brundage,Julian Marquez,Cody Brundage,9,6,11,6,2025-03-01,74.0,72.0,72.0,8.800000,7.400000,79
7856,Manel Kape,Asu Almabayev,Manel Kape,21,3,21,7,2025-03-01,64.0,65.0,68.0,4.066667,2.666667,115


In [142]:
import pandas as pd
odds_stats = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\entire_odds_stats_v2.csv')
print(odds_stats.dropna().shape)
single_event = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_singe_event_features.csv')

rolling_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_new_rolling.csv')

(5092, 150)


In [144]:
with pd.option_context('display.max_info_columns', 1000):
    rolling_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7955 entries, 0 to 7954
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   control_pr_red               7179 non-null   float64
 1   td_pr_red                    7179 non-null   float64
 2   sub_att_pr_red               7179 non-null   float64
 3   sigstrikes_pm_red            7179 non-null   float64
 4   head_strikes_pm_red          7179 non-null   float64
 5   leg_strikes_pm_red           7179 non-null   float64
 6   body_strikes_pm_red          7179 non-null   float64
 7   clinch_strikes_pm_red        7179 non-null   float64
 8   ground_strikes_pm_red        7179 non-null   float64
 9   kd_pr_red                    7179 non-null   float64
 10  reverse_pr_red               7179 non-null   float64
 11  sigstrikes_absorbed_pm_red   7179 non-null   float64
 12  red_kd                       7944 non-null   float64
 13  red_sig_str_landed

In [137]:
with pd.option_context('display.max_info_columns', 1000):
    single_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7944 entries, 0 to 7943
Data columns (total 97 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   7944 non-null   int64  
 1   event_name                   7944 non-null   object 
 2   event_date                   7944 non-null   object 
 3   event_location               7944 non-null   object 
 4   fight_url                    7944 non-null   object 
 5   red_fighter                  7944 non-null   object 
 6   blue_fighter                 7944 non-null   object 
 7   red_record                   7944 non-null   object 
 8   blue_record                  7944 non-null   object 
 9   dob_red                      7944 non-null   object 
 10  dob_blue                     7944 non-null   object 
 11  height_red                   7943 non-null   float64
 12  height_blue                  7942 non-null   float64
 13  reach_red         

In [143]:
with pd.option_context('display.max_info_columns', 1000):
    odds_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7954 entries, 0 to 7953
Data columns (total 150 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    reach_diff                   7223 non-null   float64
 1    age_diff                     7911 non-null   float64
 2    total_bonus_red              7176 non-null   float64
 3    total_bonus_blue             6348 non-null   float64
 4    total_bonus_diff             6020 non-null   float64
 5    red_td_accuracy_avg          7176 non-null   float64
 6    blue_td_accuracy_avg         6348 non-null   float64
 7    red_td_defense_avg           7176 non-null   float64
 8    blue_td_defense_avg          6348 non-null   float64
 9    red_td_landed_total          7176 non-null   float64
 10   blue_td_landed_total         6348 non-null   float64
 11   red_td_defended_total        7176 non-null   float64
 12   blue_td_defended_total       6348 non-null   float64
 13   t

In [126]:
odds_stats.dropna().shape

(3940, 151)

In [145]:
upcoming_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\upcoming_odds_stats_2025-08-09.csv')
with pd.option_context('display.max_info_columns', 1000):

    upcoming_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 150 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    reach_diff                   11 non-null     float64
 1    age_diff                     11 non-null     float64
 2    total_bonus_red              11 non-null     int64  
 3    total_bonus_blue             11 non-null     int64  
 4    total_bonus_diff             11 non-null     int64  
 5    red_td_accuracy_avg          11 non-null     float64
 6    blue_td_accuracy_avg         11 non-null     float64
 7    red_td_defense_avg           11 non-null     float64
 8    blue_td_defense_avg          11 non-null     float64
 9    red_td_landed_total          11 non-null     float64
 10   blue_td_landed_total         11 non-null     float64
 11   red_td_defended_total        11 non-null     float64
 12   blue_td_defended_total       11 non-null     float64
 13   total

In [68]:
odds_history_prev =  pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\entire_odds_history.csv')

recent_odds = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\recent_odds.csv')
full_odds = pd.concat([odds_history_prev, recent_odds])
full_odds['event_date'] = pd.to_datetime(full_odds['event_date'])
full_odds = full_odds.sort_values(by='event_date').reset_index(drop=True)

full_odds.shape

(8026, 13)

In [45]:
full_odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8026 entries, 0 to 8025
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0.1    7946 non-null   float64       
 1   Unnamed: 0      7946 non-null   float64       
 2   blue_fighter    8026 non-null   object        
 3   open_blue       8026 non-null   int64         
 4   close1_blue     8026 non-null   int64         
 5   close2_blue     8026 non-null   int64         
 6   red_fighter     8026 non-null   object        
 7   open_red        8026 non-null   int64         
 8   close1_red      8026 non-null   int64         
 9   close2_red      8026 non-null   int64         
 10  event_date      8013 non-null   datetime64[ns]
 11  og_blue_name    8026 non-null   object        
 12  og_red_fighter  7695 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(6), object(4)
memory usage: 815.3+ KB


In [ ]:
weird_odds = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\look_at_odds.csv')

with pd.option_context('display.max_info_columns', 1000):

    weird_odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8030 entries, 0 to 8029
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         8030 non-null   int64 
 1   blue_fighter       8030 non-null   object
 2   open_blue          8030 non-null   int64 
 3   close1_blue        8030 non-null   int64 
 4   close2_blue        8030 non-null   int64 
 5   red_fighter        8030 non-null   object
 6   open_red           8030 non-null   int64 
 7   close1_red         8030 non-null   int64 
 8   close2_red         8030 non-null   int64 
 9   event_date         8030 non-null   object
 10  og_blue_name       8030 non-null   object
 11  og_red_fighter     7698 non-null   object
 12  date               8030 non-null   object
 13  red_clean          8030 non-null   object
 14  blue_clean         8030 non-null   object
 15  red_fighter_odds   8030 non-null   object
 16  blue_fighter_odds  8030 non-null   object


In [104]:
from datetime import datetime

target_date = '2025-07-19'

# Option 1: Compare using datetime.date
weird_odds['date'] = pd.to_datetime(weird_odds['date'])
matches = weird_odds[weird_odds['date'].dt.date == datetime.strptime(target_date, "%Y-%m-%d").date()]

# Option 2: Compare using string (works if format matches exactly)
# matches = df[df['event_date'] == pd.to_datetime(target_date)]

print(matches[['red_clean', 'blue_clean', 'date']])

           red_clean      blue_clean       date
7981  marvin vettori   brendan allen 2025-07-19
7982    max holloway  dustin poirier 2025-07-19
7983    max holloway  dustin poirier 2025-07-19
7984    max holloway  dustin poirier 2025-07-19
7985     adam fugitt   islam dulatov 2025-07-19
7986     adam fugitt   islam dulatov 2025-07-19
7987  marvin vettori   brendan allen 2025-07-19


In [97]:
weird_odds[['red_clean', 'blue_clean', 'open_red', 'open_blue', 'date']].tail()

,red_clean,blue_clean,open_red,open_blue,date
8025,julius walker,raffael cerqueira,-649,400,2025-08-09
8026,miles johns,jean matsumoto,240,-305,2025-08-09
8027,roman dolideze,anthony hernandez,275,-350,2025-08-09
8028,steve erceg,ode osbourne,-385,295,2025-08-09
8029,uros medic,gilbert urbina,-300,240,2025-08-09


In [112]:
target_date = '2025-07-12'

# Option 1: Compare using datetime.date
odds_stats['date'] = pd.to_datetime(odds_stats['date'])
matches = odds_stats[odds_stats['date'].dt.date == datetime.strptime(target_date, "%Y-%m-%d").date()]

# Option 2: Compare using string (works if format matches exactly)
# matches = df[df['event_date'] == pd.to_datetime(target_date)]


matches[['date', 'red_fighter', 'blue_fighter', 'open_red', 'open_blue']].tail(45)

,date,red_fighter,blue_fighter,open_red,open_blue
7902,2025-07-12,jake matthews,chidi njokuani,105.0,-138.0
7903,2025-07-12,mitch ramirez,mike davis,600.0,-900.0
7904,2025-07-12,fatima kline,melissa martinez,-800.0,450.0
7905,2025-07-12,kennedy nzechukwu,valter walker,-220.0,180.0
7906,2025-07-12,lauren murphy,eduarda moura,240.0,-350.0
7907,2025-07-12,max griffin,chris curtis,200.0,-275.0
7908,2025-07-12,stephen thompson,gabriel bonfim,310.0,-400.0
7909,2025-07-12,vitor petrino,austen lane,-400.0,275.0
7910,2025-07-12,nate landwehr,morgan charriere,190.0,-230.0
7911,2025-07-12,calvin kattar,steve garcia,110.0,-150.0


In [113]:
odds_stats.dropna().shape

(4170, 105)

In [118]:
entire_odds_stats_path = r'C:\Users\jcmar\my_files\SportsBetting\data\entire_odds_stats.csv'

old_df = pd.read_csv(entire_odds_stats_path)
print(old_df.shape)

(6796, 130)


In [127]:
with pd.option_context('display.max_info_columns', 1000):
    old_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6796 entries, 0 to 6795
Data columns (total 130 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    Unnamed: 0                   6796 non-null   int64  
 1    event_age                    6796 non-null   int64  
 2    winner                       6796 non-null   int64  
 3    win_pct_red                  6796 non-null   float64
 4    win_pct_blue                 6796 non-null   float64
 5    num_fights_blue              6796 non-null   float64
 6    num_fights_red               6796 non-null   float64
 7    num_wins_red                 6796 non-null   float64
 8    num_wins_blue                6796 non-null   float64
 9    num_losses_red               6796 non-null   float64
 10   num_losses_blue              6796 non-null   float64
 11   months_since_red             6242 non-null   float64
 12   months_since_blue            5577 non-null   float64
 13   k

In [123]:
set(old_df.columns) - set(rolling_df.columns)

{'Unnamed: 0',
 'age_diff',
 'avg_td_accuracy_diff',
 'blue_clean',
 'blue_elo',
 'blue_fighter_odds',
 'blue_fighter_stats',
 'blue_glicko',
 'blue_glicko_rd',
 'blue_kd_total',
 'blue_lose_streak',
 'blue_sigstrike_accuracy',
 'blue_sigstrike_defense',
 'blue_td_accuracy_avg',
 'blue_td_defended_total',
 'blue_td_defense_avg',
 'blue_td_landed_total',
 'blue_win_streak',
 'body_strikes_pm_diff',
 'clinch_strikes_pm_diff',
 'close1_blue',
 'close1_red',
 'close2_blue',
 'close2_red',
 'control_pr_diff',
 'date',
 'decision_wins_blue',
 'decision_wins_red',
 'elo_diff',
 'glicko_diff',
 'glicko_rd_diff',
 'ground_strikes_pm_diff',
 'head_strikes_pm_diff',
 'kd_pr_diff',
 'ko_wins_blue',
 'ko_wins_red',
 'leg_strikes_pm_diff',
 'math_blue',
 'math_red',
 'months_since_blue',
 'months_since_red',
 'num_fights_blue',
 'num_fights_diff',
 'num_fights_red',
 'num_losses_blue',
 'num_losses_red',
 'num_wins_blue',
 'num_wins_red',
 'og_blue_name',
 'og_red_fighter',
 'open_blue',
 'open_red'

In [119]:
set(old_df.columns) - set(odds_stats.columns)

{'Unnamed: 0',
 'blue_age',
 'blue_clean',
 'blue_sigstrike_accuracy',
 'blue_sigstrike_defense',
 'body_strikes_pm_blue',
 'body_strikes_pm_red',
 'clinch_strikes_pm_blue',
 'clinch_strikes_pm_red',
 'control_pr_blue',
 'control_pr_red',
 'glicko_diff',
 'glicko_rd_diff',
 'ground_strikes_pm_blue',
 'ground_strikes_pm_red',
 'head_strikes_pm_blue',
 'head_strikes_pm_red',
 'height_blue',
 'height_red',
 'kd_pr_blue',
 'kd_pr_red',
 'leg_strikes_pm_blue',
 'leg_strikes_pm_red',
 'og_blue_name',
 'og_red_fighter',
 'reach_blue',
 'reach_red',
 'red_age',
 'red_clean',
 'red_sigstrike_accuracy',
 'red_sigstrike_defense',
 'reverse_pr_blue',
 'reverse_pr_red',
 'sig_strike_accuracy_diff',
 'sig_strike_defense_diff',
 'sigstrikes_absorbed_pm_blue',
 'sigstrikes_absorbed_pm_red',
 'sigstrikes_pm_blue',
 'sigstrikes_pm_red',
 'sub_att_pr_blue',
 'sub_att_pr_red',
 'td_defense_diff',
 'td_pr_blue',
 'td_pr_red',
 'total_damage_blue',
 'total_damage_diff',
 'total_damage_red',
 'total_landed_b

In [108]:
matches = full_odds[full_odds['red_fighter'].str.contains("kevin holland", case=False, na=False)]
print(matches[['event_date', 'red_fighter', 'blue_fighter', 'open_red', 'open_blue']])

     event_date    red_fighter          blue_fighter  open_red  open_blue
3576 2017-06-02  Kevin Holland      Curtis Millender       125       -165
4318 2018-11-24  Kevin Holland         John Phillips      -270        190
4499 2019-03-30  Kevin Holland    Gerald Meerschaert      -265        185
4611 2019-06-22  Kevin Holland    Alessio Di Chirico      -195        155
5051 2020-05-30  Kevin Holland      Daniel Rodriguez      -195        155
5053 2020-05-30  Kevin Holland      Daniel Rodriguez      -195        155
5056 2020-05-30  Kevin Holland      Daniel Rodriguez      -195        155
5173 2020-08-08  Kevin Holland       Joaquin Buckley      -275        235
5240 2020-09-19  Kevin Holland        Darren Stewart      -265        225
5308 2020-10-31  Kevin Holland     Charlie Ontiveros      -800        550
5770 2021-10-02  Kevin Holland          Kyle Daukaus      -110       -110
6020 2022-03-05  Kevin Holland         Alex Oliveira      -200        170
6333 2022-09-10  Kevin Holland      Da

In [ ]:
full_odds[['red_fighter', 'blue_fighter', 'open_red', 'open_blue', 'event_date']].tail(40)

In [41]:
odds_history['event_date'].tail()

7941    2025-06-21
7942    2025-06-29
7943    2025-06-21
7944    2025-07-13
7945    2025-06-29
Name: event_date, dtype: object

In [ ]:
        stats_df = stats_df.loc[:, ~stats_df.columns.str.contains('^Unnamed')]
        past_event_stats = single_event_features(stats_df.copy())
        past_event_stats.to_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_singe_event_features.csv', index=False)
        past_event_stats = past_event_stats.loc[:, ~past_event_stats.columns.str.contains('^Unnamed')]
        print(past_event_stats.shape, 'past')
        
        exclude_cols = ['red_fighter','blue_fighter']
        stats_cols = [col for col in past_event_stats.columns if col not in exclude_cols] #get NaNs for all cols EXCEPT fighter names 

        upcoming_fighters = upcoming_event_features(upcoming_stats)
        upcoming_single_event = pd.DataFrame(None, index=range(upcoming_fighters.shape[0]), columns=stats_cols)
        concat_single_event = pd.concat([upcoming_fighters[exclude_cols], upcoming_single_event], axis=1, ignore_index=True)
        concat_single_event.columns = past_event_stats.columns #this includes fighter names, they should be equal

        scraped_columns = ['red_fighter','blue_fighter', 'weight_class','event_date', 'reach_red','reach_blue',
                                                        'height_red','height_blue','red_age','blue_age','event_age']
        
        concat_single_event[scraped_columns] = upcoming_fighters[scraped_columns]
        concat_single_event['event_date'] = pd.to_datetime(upcoming_stats['event_date']) 

        #upcoming just has NaNs for stats 
        combined_df = pd.concat([concat_single_event, past_event_stats], axis=0).reset_index(drop=True)

        rolling_df = apply_rolling_stats(combined_df) #sort here 
        rolling_fp = r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_new_rolling.csv'
        rolling_df.to_csv(rolling_fp, index=False)

        total_df = non_rolling_stats(rolling_df)
        total_df.to_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\new_combined.csv', index=False)

        merged_df = self.standardized_merge(total_df, odds_df)
        merged_df = merged_df.sort_values(by='date', ascending=True).reset_index(drop=True)
        
        merged_df = merged_df.iloc[:-upcoming_fighters.shape[0], :]

        upcoming_df = total_df.iloc[-upcoming_fighters.shape[0]:, :]

        return merged_df, upcoming_df